<div style="float:left;font-size:20px;">
    <h1>Interpolation ND Error</h1>
</div><div style="float:right;"><img src="../assets/banner.jpg"></div>

Source: https://nbviewer.jupyter.org/github/pierre-haessig/stodynprog/blob/master/stodynprog/linear_interp_benchmark.ipynb

In [1]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [2]:
# Tweak how images are plotted with imshow
mpl.rcParams['image.interpolation'] = 'none' # no interpolation
mpl.rcParams['image.origin'] = 'lower' # origin at lower left corner
mpl.rcParams['image.cmap'] = 'RdBu_r'

In [3]:
from scipy.interpolate import (
    LinearNDInterpolator, RectBivariateSpline,
    RegularGridInterpolator)

In [4]:
def gridpoints(n, a=-1, b=1):
    """Returns Chebyshev nodes of specified dimensions.
    
    int n: Number of Chebyshev nodes
    float a: Interval minimum
    float b: Interval maximum
    
    Further reading: https://en.wikipedia.org/wiki/Chebyshev_nodes
    """
    x_i = 0.5*(b + a) + 0.5*(b - a) * np.cos( np.pi * (np.arange(1, n + 1) - 0.5)/n)
    return np.array(sorted(x_i)  )

In [5]:
def f_3d(x,y,z):
    '''a function with 3D input to interpolate on [0,1]'''
    twopi = 2*pi
    return np.sin(x*2*pi)*np.sin(y*2*pi)*np.sin(z*2*pi)

# quick check :
f_3d(0.25,0.25,0.25)

1.0

In [37]:
# Generate train set
Ndata = 3
xgrid = np.linspace(0,1, Ndata)
ygrid = np.linspace(0,1, Ndata+1) # use a slighly different size to check differences
zgrid = np.linspace(0,1, Ndata+2)


xgrid = gridpoints(Ndata, 0, 1)
ygrid = gridpoints(Ndata+1, 0, 1)
zgrid = gridpoints(Ndata+2, 0, 1)


# Reshape arguments specify dimensions in x, y, z. -1 infers length from array.
# (-1, 1, 1) -> Shape (len(arr), 1, 1)
values = f_3d(xgrid.reshape(-1,1,1), ygrid.reshape(1,-1,1), zgrid)

# Train interpolation
f_3d_interp = RegularGridInterpolator((xgrid, ygrid, zgrid), values)
print(f'Grid range:\nX = ({min(xgrid)}, {max(xgrid)})\nY = ({min(ygrid)}, {max(ygrid)})\nZ = ({min(zgrid)}, {max(zgrid)}')

Grid range:
X = (0.06698729810778065, 0.9330127018922194)
Y = (0.03806023374435663, 0.9619397662556434)
Z = (0.024471741852423234, 0.9755282581475768


In [ ]:
# Evaluate performance
n_points = np.arange(3, 100) # Range of points to evaluate accuracy over

# Evaluation points
Ninterp = 101
xinterp = np.linspace(0.11,0.89, Ninterp)
yinterp = np.linspace(0.11,0.89, Ninterp) # use a slighly different size to check differences
zinterp = np.linspace(0.11,0.89, Ninterp) # small dimension to avoid size explosion

points_x, points_y, points_z = np.broadcast_arrays(xinterp.reshape(-1,1,1), yinterp.reshape(1,-1,1), zinterp)
coord = np.vstack((points_x.flatten(), # a weird formula !
                   points_y.flatten(),
                   points_z.flatten()
                   ))

errors_lin, errors_cubic, errors_bary = [], [], []
errors_lin_cheby, errors_cubic_cheby, errors_bary_cheby = [], [], []
for n in n_points:

    xgrid = np.linspace(0.1, 0.9, n)
    ygrid = np.linspace(0.1, 0.9, n)
    zgrid = np.linspace(0.1, 0.9, n)
    values = f_3d(xgrid.reshape(-1,1,1), ygrid.reshape(1,-1,1), zgrid)

    xgrid_cheby = gridpoints(n, 0, 1)
    ygrid_cheby = gridpoints(n, 0, 1)
    zgrid_cheby = gridpoints(n, 0, 1)
    values_cheby = f_3d(xgrid_cheby.reshape(-1,1,1), ygrid_cheby.reshape(1,-1,1), zgrid_cheby)

    # Train interpolation
    f_3d_interp = RegularGridInterpolator((xgrid, ygrid, zgrid), values)
    f_3d_interp_cheby = RegularGridInterpolator((xgrid_cheby, ygrid_cheby, zgrid_cheby), values_cheby)
    
    value_interp = f_3d_interp(coord.T).reshape(len(xinterp), len(yinterp), len(zinterp))
    value_interp_cheby = f_3d_interp_cheby(coord.T).reshape(len(xinterp), len(yinterp), len(zinterp))
    
    value_true = f_3d(xinterp.reshape(-1,1,1), yinterp.reshape(1,-1,1), zinterp)
    value_error = np.abs(value_interp - value_true)
    value_error_cheby = np.abs(value_interp_cheby - value_true)
    errors_bary.append(np.max(np.abs(value_error)))
    errors_bary_cheby.append(np.max(np.abs(value_error_cheby)))

In [ ]:
#plt.plot(n_points, errors_lin_cheby, color='r', alpha=0.4, label='Linear Cheb')
#plt.plot(n_points, errors_cubic_cheby, color='b', alpha=0.4, label='Cubic Cheb')
plt.plot(n_points, errors_bary_cheby, color='g', alpha=0.4, label='Barycentric Cheb')
#plt.plot(n_points, errors_lin, color='r', alpha=0.8, label='Linear', ls='--')
#plt.plot(n_points, errors_cubic, color='b', alpha=0.8, label='Cubic', ls='--')
plt.plot(n_points, errors_bary, color='g', alpha=0.8, label='Barycentric', ls='--')
#plt.plot(n_points, error_theory, color='y', alpha=0.8, label='Theory', ls='--')
plt.xlabel('N points')
plt.ylabel('Absolute error')
plt.yscale('log')
plt.legend();

## RegularGridInterpolator (scipy.interpolate)

Appears to be the best of these options

In [15]:
# Prepare the coordinates to evaluate the array on :
points_x, points_y, points_z = np.broadcast_arrays(xinterp.reshape(-1,1,1), yinterp.reshape(1,-1,1), zinterp)
coord = np.vstack((points_x.flatten(), # a weird formula !
                   points_y.flatten(),
                   points_z.flatten()
                   ))
coord.shape

(3, 5005000)

In [16]:
%%timeit # Interpolate
f_3d_interp_res = f_3d_interp(coord.T).reshape(len(xinterp), len(yinterp), len(zinterp))

2.87 s ± 21.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Analysis

[documentation](http://scipy.github.io/devdocs/generated/scipy.interpolate.RegularGridInterpolator.html)

actual interpolation code: https://github.com/scipy/scipy/blob/master/scipy/interpolate/interpolate.py#L1577 (pure Python, no Cython involved!)

API:

```RegularGridInterpolator(points, values, [...])```

- `points`: tuple of ndarray of float, with shapes _(m1, ), ..., (mn, )_.
→ The points defining the regular grid in n dimensions.
- `value`: array_like, shape _(m1, ..., mn, ...)_.
→ The data on the regular grid in n dimensions.

**Performance**
- 0.022 ms for instanciation, 116 ms for evaluation (1Mpts)
- in 1.8 s in 3D (5 Mpts)